In [11]:
# -*- coding:utf-8 -*-
import pandas as pd
import subprocess
import pydotplus
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE      # Recursive Feature Elimination
from sklearn.feature_selection import chi2
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import pipeline

# 数据导入
xls_file = pd.ExcelFile('alliance.xls')
# print(xls_file.sheet_names)
df = xls_file.parse('sheet1')
df_target = df['result']

In [12]:
# 对范畴类变量进行编码
for column in ['uristem','uriquery','http_status']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns]=dummies

# 去掉已编码的原属性
df.drop(['domain','uristem','uriquery','http_status'], axis=1, inplace=True)
#df.ix[:,df.columns!=999]

In [13]:
# 拆分数据集为训练集和测试集
train, test = train_test_split(df, test_size = 0.2)   # doing split of training and testing
train_data, train_target= train.ix[:, train.columns != 'result'], train['result']
test_data, test_target = test.ix[:, test.columns != 'result'], test['result']
print(train_data.shape)
print(train_target.shape)

(1244, 19)
(1244,)


In [14]:
# 特征选择 + 分类器 + 带入管道
# # Create the RFE object and rank each pixel
# svc = SVC(kernel="linear", C=1)
# rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
# rfe.fit(X, y)
selector = SelectKBest(chi2, k=5)
dt = DecisionTreeClassifier(min_samples_split=20, random_state=15)
pipe = pipeline.make_pipeline(selector, dt)
final_pipe = pipe.fit(train_data, train_target)

In [15]:
# 交叉验证
train_scores = cross_val_score(final_pipe , test_data, test_target, cv=7,scoring='accuracy')
print(train_scores)
#test_scores = cross_val_score(clf, test_data, test_target, cv=5,scoring='accuracy')
#print(scores)

[ 0.8         0.75555556  0.77777778  0.75555556  0.82222222  0.70454545
  0.74418605]
